In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
from matplotlib.collections import LineCollection
import pickle
from pickle import UnpicklingError
import os
from os import path
import warnings
import arviz as az

warnings.filterwarnings("ignore")

class Config:
    sim_ring_tracks = 34
    sim_ring_radius = 128
    sim_ring_time = 6000
    sim_ring_init_speed = 10.22
    sim_veh_length = 5
    dt = 0.5
    frame_rate = 2

np.random.seed(1116)

def a_IDM(VMAX, DSAFE, TSAFE, AMAX, AMIN, DELTA, s, vt, dv):
    sn = DSAFE + np.max((vt * TSAFE + vt * dv / (2 * np.sqrt(AMAX * AMIN)), 0))
    a = AMAX * (1 - (vt / VMAX) ** DELTA - (sn / s) ** 2)
    return a

def extract_population_parameters(trace):
    posterior = trace.posterior
    
    log_mu_vmax = posterior['log_mu_vmax'].mean(dim=['chain', 'draw']).values
    log_mu_dsafe = posterior['log_mu_dsafe'].mean(dim=['chain', 'draw']).values
    log_mu_tsafe = posterior['log_mu_tsafe'].mean(dim=['chain', 'draw']).values
    log_mu_amax = posterior['log_mu_amax'].mean(dim=['chain', 'draw']).values
    log_ratio_mu = posterior['log_ratio_mu'].mean(dim=['chain', 'draw']).values
    
    mu_vmax = np.exp(log_mu_vmax) * 25
    mu_dsafe = np.exp(log_mu_dsafe) * 2
    mu_tsafe = np.exp(log_mu_tsafe) * 1.6
    mu_amax = np.exp(log_mu_amax) * 1.5
    ratio_mu = np.exp(log_ratio_mu)
    mu_amin = ratio_mu * mu_amax * 1.5
    
    chol = posterior['chol'].mean(dim=['chain', 'draw']).values
    
    if chol.ndim == 1:
        n_params = 5
        chol_matrix = np.zeros((n_params, n_params))
        idx = 0
        for i in range(n_params):
            for j in range(i+1):
                chol_matrix[i, j] = chol[idx]
                idx += 1
        chol = chol_matrix
    
    rho_mu = posterior['rho_mu'].mean(dim=['chain', 'draw']).values
    rho_raw_std = 0.1
    
    s_a_population = []
    for i in range(4):
        s_a_key = f's_a_{i}'
        if s_a_key in posterior:
            s_a = posterior[s_a_key].mean(dim=['chain', 'draw']).values
            s_a_population.append(s_a)
    
    s_a_mean = np.mean(s_a_population) if s_a_population else 0.1
    s_a_std = np.std(s_a_population) if s_a_population else 0.05
    
    population_params = {
        'mu': np.array([mu_vmax, mu_dsafe, mu_tsafe, mu_amax, mu_amin]),
        'chol': chol,
        'rho_mu': rho_mu,
        'rho_std': rho_raw_std,
        's_a_mean': s_a_mean,
        's_a_std': s_a_std
    }
    
    
    return population_params

def generate_vehicle_parameters(population_params, n_vehicles):
    mu = population_params['mu']
    chol = population_params['chol']
    
    n_params = 5
    z = np.random.normal(0, 1, (n_vehicles, n_params))
    individual_effects = z @ chol.T
    
    params = np.zeros((n_vehicles, 5))
    for i in range(n_vehicles):
        for j in range(5):
            params[i, j] = mu[j] * np.exp(individual_effects[i, j])
    
    params[:, 0] = np.clip(params[:, 0], 15, 35)
    params[:, 1] = np.clip(params[:, 1], 0.5, 4)
    params[:, 2] = np.clip(params[:, 2], 0.5, 4)
    params[:, 3] = np.clip(params[:, 3], 0.5, 10)
    params[:, 4] = np.clip(params[:, 4], 0.5, 10)
    
    rho = np.zeros((n_vehicles, 1))
    for i in range(n_vehicles):
        rho[i, 0] = population_params['rho_mu'][0] + np.random.normal(0, population_params['rho_std'])
        rho[i, 0] = np.clip(rho[i, 0], 0.1, 0.96)
    
    s_a_list = []
    for i in range(n_vehicles):
        s_a = np.random.normal(population_params['s_a_mean'], population_params['s_a_std'])
        s_a = max(s_a, 0.001)
        s_a_list.append(s_a)
    
    
    return params, rho, s_a_list

def your_ring_scenarios(trace, sim_method='your_AR'):
    n_vehicles = Config.sim_ring_tracks
    
    population_params = extract_population_parameters(trace)
    

    params, rho, s_a_list = generate_vehicle_parameters(population_params, n_vehicles)
    
    
    tracks = [None] * n_vehicles
    init_pos_theta = np.linspace(0, 2 * np.pi, n_vehicles + 1)
    R = Config.sim_ring_radius
    N = Config.sim_ring_time
    
    for id in range(n_vehicles):
        tracks[id] = {
            'pos_s': np.zeros(N),
            'pos_theta': np.zeros(N), 
            'a': np.zeros(N),
            'v': np.zeros(N),
            'a_eps': np.zeros(N)
        }
    
    a_eps_prev = np.zeros(n_vehicles)
    

    for t in range(N - 1):
        for id in range(n_vehicles):
            if id == n_vehicles - 1:
                id_pre = 0
            else:
                id_pre = id + 1
            
            if t == 0:
                tracks[id]['pos_s'][0] = init_pos_theta[id] * R
                tracks[id]['pos_theta'][0] = tracks[id]['pos_s'][0] / R
                tracks[id]['v'][0] = Config.sim_ring_init_speed
                a_eps_prev[id] = 0
            
            s = tracks[id_pre]['pos_s'][t] - tracks[id]['pos_s'][t] - Config.sim_veh_length
            dv = tracks[id]['v'][t] - tracks[id_pre]['v'][t]
            v = tracks[id]['v'][t]
            
            if s < 0:
                s += Config.sim_ring_radius * 2 * np.pi
            
            if sim_method == 'your_AR':
                a_idm = a_IDM(
                    params[id, 0], params[id, 1], params[id, 2], 
                    params[id, 3], params[id, 4], 4, s, v, dv
                )
                
                if t >= 1:
                    ar_correction = rho[id, 0] * a_eps_prev[id]
                else:
                    ar_correction = 0
                
                new_eps = np.random.normal(0, s_a_list[id])
                a_eps = ar_correction + new_eps
                
                a_total = a_idm + a_eps
                a_total = np.max((a_total, -10))
                
                a_eps_prev[id] = a_eps
                tracks[id]['a_eps'][t] = a_eps
            
            v_new = np.max((v + a_total * Config.dt, 0))
            dx = 0.5 * (v_new + v) * Config.dt
            
            tracks[id]['v'][t + 1] = v_new
            tracks[id]['a'][t] = a_total
            tracks[id]['pos_s'][t + 1] = (tracks[id]['pos_s'][t] + dx) % (2 * np.pi * R)
            tracks[id]['pos_theta'][t + 1] = tracks[id]['pos_s'][t + 1] / R
        
        if (t + 1) % 100 == 0:
            print(f"Simulation progress: {t + 1}/{N}")
    
    return tracks

def fundamental_area(tracks, L, T):
    T_all = tracks[0]['a'].shape[0]
    N = round(T_all / T)
    circumference = Config.sim_ring_radius * 2 * np.pi
    N_s = len(range(0, int(circumference) - L, L))
    
    if N_s <= 0:
        N_s = 1
    
    density = np.zeros((N - 1, N_s))
    speed = np.zeros((N - 1, N_s))

    for count, lower in enumerate(range(0, int(circumference) - L, L)):
        upper = lower + L
        for tt in range(N - 1):
            d_all = 0
            t_all = 0
            for id in range(len(tracks)):
                for t in range(T * tt, T * (tt + 1)):
                    x = tracks[id]['pos_s'][t]
                    v = tracks[id]['v'][t]
                    if x < upper and x > lower:
                        d_all += v * Config.dt
                        t_all += Config.dt
            density[tt, count] = t_all / (L * T * Config.dt)
            if t_all == 0:
                speed[tt, count] = 0
            else:
                speed[tt, count] = d_all / t_all

    flow = density * speed
    return speed * 3.6, density * 1000, flow * 3600

def plot_speed_density(tracks, savefig=True):
    plt.rcParams['text.usetex'] = False
    plt.rcParams["font.family"] = "Arial"
    matplotlib.rcParams['font.size'] = 22
    matplotlib.rcParams['font.family'] = 'Arial'
    plt.rcParams['mathtext.fontset'] = 'cm'

    L = 50
    T = int(60 / Config.dt)

    speed, density, flow = fundamental_area(tracks, L, T)

    valid_mask = (density > 0) & (speed > 0) & (flow > 0)
    density_valid = density[valid_mask]
    speed_valid = speed[valid_mask]

    fig = plt.figure(figsize=(7, 5))
    plt.scatter(density_valid, speed_valid, marker='s', color='b', alpha=0.6)
    plt.xlabel('Density (veh/km)')
    plt.ylabel('Speed (km/h)')
    plt.title('Speed-Density Relationship')
    plt.tight_layout()
    if savefig:
        os.makedirs('../Figs', exist_ok=True)
        fig.savefig('../Figs/Sim_vk_fundamental_diagram.pdf', dpi=300)
    plt.show()

sim_method = 'your_AR'
tracks = your_ring_scenarios(trace, sim_method='your_AR')
plot_speed_density(tracks, savefig=True)

In [ ]:
class Config:
    sim_ring_tracks = 36
    sim_ring_radius = 128
    sim_ring_time = 6000
    sim_ring_init_speed = 9.6
    sim_veh_length = 5
    dt = 0.5
    frame_rate = 2

np.random.seed(1116)

def a_IDM(VMAX, DSAFE, TSAFE, AMAX, AMIN, DELTA, s, vt, dv):
    sn = DSAFE + np.max((vt * TSAFE + vt * dv / (2 * np.sqrt(AMAX * AMIN)), 0))
    a = AMAX * (1 - (vt / VMAX) ** DELTA - (sn / s) ** 2)
    return a

def extract_population_parameters(trace):
    posterior = trace.posterior
    
    log_mu_vmax = posterior['log_mu_vmax'].mean(dim=['chain', 'draw']).values
    log_mu_dsafe = posterior['log_mu_dsafe'].mean(dim=['chain', 'draw']).values
    log_mu_tsafe = posterior['log_mu_tsafe'].mean(dim=['chain', 'draw']).values
    log_mu_amax = posterior['log_mu_amax'].mean(dim=['chain', 'draw']).values
    log_ratio_mu = posterior['log_ratio_mu'].mean(dim=['chain', 'draw']).values
    
    mu_vmax = np.exp(log_mu_vmax) * 25
    mu_dsafe = np.exp(log_mu_dsafe) * 2
    mu_tsafe = np.exp(log_mu_tsafe) * 1.6
    mu_amax = np.exp(log_mu_amax) * 1.5
    ratio_mu = np.exp(log_ratio_mu)
    mu_amin = ratio_mu * mu_amax * 1.5
    
    chol = posterior['chol'].mean(dim=['chain', 'draw']).values
    
    if chol.ndim == 1:
        n_params = 5
        chol_matrix = np.zeros((n_params, n_params))
        idx = 0
        for i in range(n_params):
            for j in range(i+1):
                chol_matrix[i, j] = chol[idx]
                idx += 1
        chol = chol_matrix
    
    rho_0_mean = posterior['rho_0'].mean(dim=['chain', 'draw']).values.mean(axis=0)
    rho_0_std = posterior['rho_0'].mean(dim=['chain', 'draw']).values.std(axis=0)
    
    rho_innovation_mean = posterior['rho_innovation'].mean(dim=['chain', 'draw']).values.mean(axis=0)
    rho_innovation_std = posterior['rho_innovation'].mean(dim=['chain', 'draw']).values.std(axis=0)
    
    s_a_population = []
    for i in range(4):
        s_a_key = f's_a_{i}'
        if s_a_key in posterior:
            s_a = posterior[s_a_key].mean(dim=['chain', 'draw']).values
            s_a_population.append(s_a)
    
    s_a_mean = np.mean(s_a_population) if s_a_population else 0.1
    s_a_std = np.std(s_a_population) if s_a_population else 0.05
    
    population_params = {
        'mu': np.array([mu_vmax, mu_dsafe, mu_tsafe, mu_amax, mu_amin]),
        'chol': chol,
        'rho_0_mean': rho_0_mean,
        'rho_0_std': rho_0_std,
        'rho_innovation_mean': rho_innovation_mean,
        'rho_innovation_std': rho_innovation_std,
        's_a_mean': s_a_mean,
        's_a_std': s_a_std
    }
    
    
    return population_params

def generate_vehicle_parameters(population_params, n_vehicles):
    mu = population_params['mu']
    chol = population_params['chol']
    
    n_params = 5
    z = np.random.normal(0, 1, (n_vehicles, n_params))
    individual_effects = z @ chol.T * 0.6
    
    params = np.zeros((n_vehicles, 5))
    for i in range(n_vehicles):
        for j in range(5):
            params[i, j] = mu[j] * np.exp(individual_effects[i, j])
    
    params[:, 0] = np.clip(params[:, 0], 15, 35)
    params[:, 1] = np.clip(params[:, 1], 0.5, 4)
    params[:, 2] = np.clip(params[:, 2], 0.5, 4)
    params[:, 3] = np.clip(params[:, 3], 0.5, 10)
    params[:, 4] = np.clip(params[:, 4], 0.5, 10)
    
    s_a_list = []
    for i in range(n_vehicles):
        s_a = np.random.normal(population_params['s_a_mean'], population_params['s_a_std'])
        s_a = max(s_a, 0.001)
        s_a_list.append(s_a)
    

    
    return params, s_a_list

def generate_dynamic_ar_coefficients(population_params, n_vehicles, total_time_steps, tau_sec=3, ar_order=1):
    dt = Config.dt
    tau = int(tau_sec / dt)
    n_blocks = int(np.ceil(total_time_steps / tau))
    
    
    rho_dynamic = np.zeros((n_vehicles, total_time_steps, ar_order))
    
    for i in range(n_vehicles):
        for lag in range(ar_order):
            current_rho = np.random.normal(population_params['rho_0_mean'][lag], population_params['rho_0_std'][lag])
            current_rho = np.clip(current_rho, 0.91, 0.999)
            
            current_innovation_std = np.random.normal(population_params['rho_innovation_mean'][lag], 
                                                     population_params['rho_innovation_std'][lag])
            current_innovation_std = max(current_innovation_std, 0.001)
            
            for block in range(n_blocks):
                start_idx = block * tau
                end_idx = min((block + 1) * tau, total_time_steps)
                
                rho_dynamic[i, start_idx:end_idx, lag] = current_rho
                
                if block < n_blocks - 1:
                    innovation = np.random.normal(0, current_innovation_std)
                    current_rho += innovation
                    current_rho = np.clip(current_rho, 0.91, 0.999)
    
    
    return rho_dynamic

def your_dynamic_ar_ring_scenarios(trace, sim_method='dynamic_AR', tau_sec=5):
    n_vehicles = Config.sim_ring_tracks
    total_time_steps = Config.sim_ring_time
    

    population_params = extract_population_parameters(trace)
    
    params, s_a_list = generate_vehicle_parameters(population_params, n_vehicles)
    

    rho_dynamic = generate_dynamic_ar_coefficients(
        population_params, n_vehicles, total_time_steps, tau_sec, ar_order=1
    )
    
    
    tracks = [None] * n_vehicles
    init_pos_theta = np.linspace(0, 2 * np.pi, n_vehicles + 1)
    R = Config.sim_ring_radius
    N = Config.sim_ring_time
    
    for id in range(n_vehicles):
        tracks[id] = {
            'pos_s': np.zeros(N),
            'pos_theta': np.zeros(N), 
            'a': np.zeros(N),
            'v': np.zeros(N),
            'a_eps': np.zeros(N),
            'rho_actual': np.zeros(N)
        }
    
    a_eps_prev = np.zeros(n_vehicles)
    
    for t in range(N - 1):
        for id in range(n_vehicles):
            if id == n_vehicles - 1:
                id_pre = 0
            else:
                id_pre = id + 1
            
            if t == 0:
                tracks[id]['pos_s'][0] = init_pos_theta[id] * R
                tracks[id]['pos_theta'][0] = tracks[id]['pos_s'][0] / R
                tracks[id]['v'][0] = Config.sim_ring_init_speed
                a_eps_prev[id] = 0
            
            s = tracks[id_pre]['pos_s'][t] - tracks[id]['pos_s'][t] - Config.sim_veh_length
            dv = tracks[id]['v'][t] - tracks[id_pre]['v'][t]
            v = tracks[id]['v'][t]
            
            if s < 0:
                s += Config.sim_ring_radius * 2 * np.pi
            
            if sim_method == 'dynamic_AR':
                a_idm = a_IDM(
                    params[id, 0], params[id, 1], params[id, 2], 
                    params[id, 3], params[id, 4], 4, s, v, dv
                )
                
                current_rho = rho_dynamic[id, t, 0]
                tracks[id]['rho_actual'][t] = current_rho
                
                if t >= 1:
                    ar_correction = current_rho * a_eps_prev[id]
                else:
                    ar_correction = 0
                
                new_eps = np.random.normal(0, s_a_list[id])
                a_eps = ar_correction + new_eps
                
                a_total = a_idm + a_eps
                a_total = np.max((a_total, -10))
                
                a_eps_prev[id] = a_eps
                tracks[id]['a_eps'][t] = a_eps
                
            else:
                a_total = a_IDM(33.3, 2.0, 1.6, 1.5, 1.67, 4, s, v, dv)
                a_total = np.max((a_total, -10))
            
            v_new = np.max((v + a_total * Config.dt, 0))
            dx = 0.5 * (v_new + v) * Config.dt
            
            tracks[id]['v'][t + 1] = v_new
            tracks[id]['a'][t] = a_total
            tracks[id]['pos_s'][t + 1] = (tracks[id]['pos_s'][t] + dx) % (2 * np.pi * R)
            tracks[id]['pos_theta'][t + 1] = tracks[id]['pos_s'][t + 1] / R
        
        if (t + 1) % 100 == 0:
            current_rhos = np.array([tracks[id]['rho_actual'][t] for id in range(n_vehicles)])
            print(f"Simulation progress: {t + 1}/{N}, Current AR coefficient mean: {current_rhos.mean():.3f}")
    
    return tracks

def fundamental_area(tracks, L, T):
    T_all = tracks[0]['a'].shape[0]
    N = round(T_all / T)
    circumference = Config.sim_ring_radius * 2 * np.pi
    N_s = len(range(0, int(circumference) - L, L))
    
    if N_s <= 0:
        N_s = 1
    
    density = np.zeros((N - 1, N_s))
    speed = np.zeros((N - 1, N_s))

    for count, lower in enumerate(range(0, int(circumference) - L, L)):
        upper = lower + L
        for tt in range(N - 1):
            d_all = 0
            t_all = 0
            for id in range(len(tracks)):
                for t in range(T * tt, T * (tt + 1)):
                    x = tracks[id]['pos_s'][t]
                    v = tracks[id]['v'][t]
                    if x < upper and x > lower:
                        d_all += v * Config.dt
                        t_all += Config.dt
            density[tt, count] = t_all / (L * T * Config.dt)
            if t_all == 0:
                speed[tt, count] = 0
            else:
                speed[tt, count] = d_all / t_all

    flow = density * speed
    return speed * 3.6, density * 1000, flow * 3600

def plot_speed_density(tracks, savefig=True):
    plt.rcParams['text.usetex'] = False
    plt.rcParams["font.family"] = "Arial"
    matplotlib.rcParams['font.size'] = 22
    matplotlib.rcParams['font.family'] = 'Arial'
    plt.rcParams['mathtext.fontset'] = 'cm'

    L = 50
    T = int(60 / Config.dt)

    speed, density, flow = fundamental_area(tracks, L, T)

    valid_mask = (density > 0) & (speed > 0) & (flow > 0)
    density_valid = density[valid_mask]
    speed_valid = speed[valid_mask]

    fig = plt.figure(figsize=(7, 5))
    plt.scatter(density_valid, speed_valid, marker='s', color='b', alpha=0.6)
    plt.xlabel('Density (veh/km)')
    plt.ylabel('Speed (km/h)')
    plt.title('Speed-Density Relationship - Dynamic AR Model')
    plt.tight_layout()
    if savefig:
        os.makedirs('../Figs', exist_ok=True)
        fig.savefig('../Figs/Sim_vk_fundamental_diagram_dynamic_AR.pdf', dpi=300)
    plt.show()

sim_method = 'dynamic_AR'
tau_sec = 5

tracks = your_dynamic_ar_ring_scenarios(trace1, sim_method='dynamic_AR', tau_sec=3)
plot_speed_density(tracks, savefig=True)